In [12]:
import sys
import os
# current_dir = os.getcwd()
# parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
# sys.path.append(parent_dir)

from SHapRAG import*
import pandas as pd
from scipy.stats import pearsonr
import time

In [ ]:
documents = [
    "The weather in Paris is sunny today.",
    "Paris is the capital of France.",
    "The sun is shining in Paris today",
    "Berlin is the capital of Germany.", # Irrelevant
    # "The Eiffel Tower is located in Paris, France.",
    # "France borders several countries including Germany.",
    # "The currency used in France is the Euro.",
    # "Paris hosted the Summer Olympics in 1900 and 1924.",
    # "Germany uses the Euro as well.", # Redundant info
    "It is cloudy in Berlin today." # Irrelevant
]
query = "What is the weather like in the capital of France?"
target_response = "Paris is sunny." # The ideal answer fragment

attributor = ShapleyAttributor(
        items=documents,
        query=query,
        target_response=target_response,
        llm_caller=compute_logprob # Use your actual llm_caller
    )

print("\n--- Computing ContextCite (Surrogate Weights) ---")
cc_weights = attributor.compute(method_name="contextcite", num_samples=16, lasso_alpha=0.01)
if cc_weights is not None: print("ContextCite Weights:", np.round(cc_weights, 4))

print("\n--- Computing Weakly Supervised Shapley (WSS) ---")
wss_values, wss_surrogate_weights = attributor.compute(method_name="wss", num_samples=16, lasso_alpha=0.01, return_weights=True)
if wss_values is not None:
    print("WSS Values:", np.round(wss_values, 4))
    print("WSS Surrogate Weights:", np.round(wss_surrogate_weights, 4))

print("\n--- Computing TMC-Shapley ---")
tmc_values = attributor.compute(method_name="tmc") # 5*n iterations
if tmc_values is not None: print("TMC Values:", np.round(tmc_values, 4))

if beta_dist: # Only if scipy is available
    print("\n--- Computing Beta-Shapley (U-shaped, more weight to ends) ---")
    # For U-shaped, alpha and beta < 1
    beta_u_values = attributor.compute(method_name="betashap", num_iterations=attributor.n_items, beta_a=0.5, beta_b=0.5)
    if beta_u_values is not None: print("BetaShap (U-shaped) Values:", np.round(beta_u_values, 4))

    print("\n--- Computing Beta-Shapley (Uniform, equivalent to standard MC) ---")
    beta_uniform_values = attributor.compute(method_name="betashap", num_iterations=attributor.n_items, beta_a=1.0, beta_b=1.0)
    if beta_uniform_values is not None: print("BetaShap (Uniform) Values:", np.round(beta_uniform_values, 4))
print("\n--- Computing Leave-One-Out (LOO) ---")
loo_values = attributor.compute(method_name="loo") 
if loo_values is not None: 
    print("LOO Values:", np.round(loo_values, 4))

print("\n--- Computing Exact Shapley ---")
exact_values = attributor.compute(method_name="exact", exact_confirm=False) # Disable confirm for n=5 demo
if exact_values is not None: print("Exact Values:", np.round(exact_values, 4))

if exact_values is not None:
    if wss_values is not None:
        mae_wss = np.mean(np.abs(exact_values - wss_values))
        print(f"\nMAE WSS vs Exact: {mae_wss:.4f}")
    if tmc_values is not None:
        mae_tmc = np.mean(np.abs(exact_values - tmc_values))
        print(f"MAE TMC vs Exact: {mae_tmc:.4f}")
    if beta_dist and beta_u_values is not None:
        mae_beta_u = np.mean(np.abs(exact_values - beta_u_values))
        print(f"MAE BetaShap (U) vs Exact: {mae_beta_u:.4f}")
    if beta_dist and beta_uniform_values is not None:
        mae_beta_uni = np.mean(np.abs(exact_values - beta_uniform_values))
        print(f"MAE BetaShap (Uni) vs Exact: {mae_beta_uni:.4f}")
    # Compare LOO to Exact if available
    if exact_values is not None and loo_values is not None:
        mae_loo = np.mean(np.abs(exact_values - loo_values))
        print(f"\nMAE LOO vs Exact: {mae_loo:.4f}")



--- Computing ContextCite (Surrogate Weights) ---
Starting contextcite (n=5, m=16)...
Computing true utilities for 16 samples...


contextcite LLM Calls:  50%|█████     | 8/16 [00:56<00:55,  6.92s/it]

In [23]:
from scipy.stats import spearmanr, pearsonr
correlation, p_value = spearmanr(exact_values, wss_values)
print(f"Spearman Correlation: {correlation:.4f}")

Spearman Correlation: 1.0000


In [ ]:
logp, p, model_out, token_probs = compute_logprob(
    query="What is the capital of Kazakhstan?",
    ground_truth_answer="Paris",
    return_token_probs=True
)

print("Log probability:", logp)
print("Total probability:", p)
print("Model generated:", model_out)